# Experiment: How good was the selection of contacted customers compared to a computed sampling set?

## Setup

In [ ]:
import sys
PROJECT_PATH = "/home/christopher_orlowicz1_vodafone_c/gershgorin"
sys.path.append(PROJECT_PATH)
%load_ext autoreload
%autoreload 2

In [ ]:
%cd $PROJECT_PATH

In [ ]:
%pip install -q -r requirements.txt
#%pip install graphilp

In [ ]:
import time

from google.cloud import bigquery
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = "retina"
import networkx as nx
import numpy as np
import pandas as pd
import pandas_gbq
import scipy
from sklearn.neighbors import kneighbors_graph

import src.db.big_query as bq
from src.db.preprocessing import Preprocessor
from src.gershgorin.bs_gda import bs_gda
from src.graph.graph import Graph
from src.graph import metrics
import src.utils.plotting as util_plt

## Build customer graph

### Load the data

Earliest possible date is **2021-12-09** (occurence of first `answer_value`s in NPS table).

In [ ]:
from_date = "2022-06-01"
to_date = "2022-06-30"

In [ ]:
# preprocess the data
data = bq.car_query_timeframe_sample(from_date, to_date, limit=1000000)
prep = Preprocessor(from_date, to_date, data=data, verbose=False)
car_df, client_ids = prep.car_df, prep.client_ids
# inspect nan columns
nan_cols = car_df.columns[car_df.isna().sum() > 0]
# remove these columns for now
car_df.drop(nan_cols, axis=1, inplace=True)

In [ ]:
n_nodes = car_df.shape[0]

### Construct graph

#### Using scikit-learn

In [ ]:
n_nodes = car_df.shape[0]
d = 0.2  # upper (directed) / lower (undirected) bound on graph density
k = np.ceil(d*n_nodes).astype(int)  # compute number of neighbors based on desired density
t = time.perf_counter()
A = kneighbors_graph(car_df, k, mode='connectivity', include_self=False, n_jobs=-1)
#A = A.maximum(A.T)  # symmetrize the matrix (but density can be significantly higher)
print(f"This took {time.perf_counter()-t:.3f} s")

#### Using FAISS

In [ ]:
from src.graph.nearest_neigbors import NearestNeighbors
np_df = car_df.to_numpy(dtype=np.float32)
knn = NearestNeighbors(k=100)
t = time.perf_counter()
knn.fit(np_df)
_, k_neighbors = knn.predict(np_df)
print(f"This took {time.perf_counter()-t:.3f} s")

In [ ]:
from scipy import sparse
A = sparse.lil_matrix((n_nodes, n_nodes), dtype=bool)
for i in range(n_nodes):
    A[i, k_neighbors[i]] = 1

In [ ]:
# directed graph: count_nonzero(A) >= n_edges >= count_nonzero(A)/2
# undirected graph: count_nonzero(A)/2 = n_edges
n_edges = A.getnnz()
metrics.density(n_nodes, n_edges, mode="directed")

## Compute sampling set

In [ ]:
sampling_budget = int(0.1*n_nodes)
graph = Graph(A)
start = time.perf_counter()
sampling_set, _ = bs_gda(graph, sampling_budget, parallel=True)
print(f"This took {time.perf_counter()-start:.3f} s")
print("Budget:", sampling_budget)
print("Sampled nodes:", len(sampling_set))

In [ ]:
# map node_id to client_id
pred_sampling_set = client_ids[sampling_set]

## Evaluate the result

### Get actually sampled customers

In [ ]:
actual_sampling_set = bq.join_car_nps(from_date, to_date).client_id

In [ ]:
np.mean(actual_sampling_set.isin(pred_sampling_set))

### Investigate neighborhood of sampled nodes